# artsy

In [5]:
import json

import requests

import credentials

### app data

In [6]:
app_name = credentials.artsy_app_name
client_id = credentials.artsy_client_id
client_secret = credentials.artsy_client_secret

### token

In [7]:
BASE_URL = 'https://api.artsy.net/api'

In [21]:
token_url = BASE_URL + '/tokens/xapp_token'
payload = {'client_id': client_id, 'client_secret': client_secret}
response = requests.post(token_url, payload)
token = response.json()['token']

## search

In [31]:
search_term = 'Andy Warhol'

In [32]:
search_url = BASE_URL + '/search'
headers = {'X-Xapp-Token': token}
payload = {'q': search_term}
response = requests.get(search_url, params=payload, headers=headers)

In [33]:
response

<Response [200]>

In [34]:
response.json()

{'_embedded': {'results': [{'_links': {'permalink': {'href': 'https://www.artsy.net/artist/andy-warhol'},
     'self': {'href': 'https://api.artsy.net/api/artists/andy-warhol'},
     'thumbnail': {'href': 'https://d32dm0rphc51dk.cloudfront.net/PFufT6nMKNwLOpPEezf4Ww/large.jpg'}},
    'description': 'Dec 5, 2015 ... Browse the best of Andy Warhol art, including paintings and artwork for sale (with \nprices), upcoming shows, and exclusive Andy Warhol articles\xa0...',
    'og_type': 'artsyinc:artist',
    'title': 'Andy Warhol Art - 1161 Artworks, Biography & Shows on Artsy',
    'type': 'Artist'},
   {'_links': {'permalink': {'href': 'https://www.artsy.net/artist/andy-warhol/works'},
     'self': {'href': 'https://api.artsy.net/api/artists/andy-warhol'},
     'thumbnail': {'href': 'https://i.embed.ly/1/display/resize?height=300&quality=95&grow=false&url=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FVFelegc1J0GFxkS2LSJdRg%2Flarger.jpg&key=a1f82558d8134f6cbebceb9e67d04980'}},
    'descrip

In [26]:
token

'JvTPWe4WsQO-xqX6Bts49p9StMhGBkSV_qt-rywOd8KTgnM2FL7UaC2PMlYKJqwkc9iCmLWSx21fG3cIikJmBWlot_AGZqFzCVFrK4UFGrFwP9NYxf1MujKzWU589ACCBlwsff8MVm39aV-Lj1wOmaTVYHeP89FMsAwZTkMnVIFKwL2WyXaoDU17pzn2HRv2hc6O0Yx5SafVyDohig8nth-mUPSPUbXkriJWOkca07E='